<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Ejercicios creados a partir de 2019-2 por Equipo Docente IIC2233. </font>
<font size='1'> Actualizados el 2023-2.</font>
</p>


# Ejercicios propuestos: Networking
## Conceptos básicos de redes de datos (*networking*)

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de **Networking**. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

### Ejercicio 1: `Bot Whatsapp OMS: funcionalidades básicas`

Si guardabas el número +41 79 893 18 92 en tu celular y le envíabas un mensaje a través de Whatsapp, entrabas en contacto con el bot oficial de la Organización Mundial de la Salud que se encargaba informar a la gente con datos actualizados del Cov-19. A través de un menú, el bot te permite seleccionar distinta información. 


En este ejercicio, simularemos el trabajo del bot de la OMS a través de Networking, donde el Bot actúa como el servidor que le provee la información del 9 de junio 2020 a las distintas personas (clientes).


En esta primera parte, el bot deberá proveer la siguiente información a un usuario: 

    [1] Casos en Chile
    [2] Casos en el Mundo
    [3] Consejos contra el Coronavirus
    [4] Reportar un nuevo caso (en 1.2 hablaremos más sobre esta opción)

La idea es que si el usuario selecciona una opción [1, 2, 3, 4] en su consola, el bot le envíe la información asociada asociada a dicha solicitud. Esa información ya se encuentra cargada y la puedes encontrar en los values del diccionario self.info del servidor. Además si el usuario ingresa 0, el bot le debe enviar la visualización del menú (también está contenida en self.info). En caso de que el usuario no envíe una opción [0, 1, 2, 3, 4], se le debe enviar el aviso: 'No seleccionaste ninguna opción. Envía 0 para recibir el menú de opciones.'

Para lograr esto, deberás implementar solamente el método `handle_command(opcion, client_socket)` del servidor. Este recibe la opción 'opcion' que le envía el usuario del socket 'client_socket' y dependiendo del valor de la opción debe enviarle de vuelta la respuesta correspondiente. A continuación se muestra el código para el bot y el usuario.



In [ ]:
import socket
import threading


class Bot:

    def __init__(self, port, host):
        print("Inicializando servidor...")

        self.host = host
        self.port = port
        self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.bind_and_listen()
        self.accept_connections()
        
        self.nuevo_caso_chile = 138846
        self.nuevo_caso_mundo = 108918
        self.casos_chile = 7039918
        self.casos_mundo = 4696
        self.actualizar_info()

    def bind_and_listen(self):

        self.socket_server.bind((self.host, self.port))
        self.socket_server.listen()
        print(f"Servidor escuchando en {self.host}:{self.port}...")

    def accept_connections(self):

        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()

    def accept_connections_thread(self):

        print("Servidor aceptando conexiones...")

        while True:
            client_socket, _ = self.socket_server.accept()
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket, ),
                daemon=True)
            listening_client_thread.start()
            

    @staticmethod
    def send(value, sock):

        stringified_value = str(value)
        msg_bytes = stringified_value.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        sock.send(msg_length + msg_bytes)
    
    def sendall(self, value):
        pass

    def listen_client_thread(self, client_socket):

        print("Servidor conectado a un nuevo cliente...")
        
        while True:
            response_bytes_length = client_socket.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(client_socket.recv(read_length))

            received = response.decode()
            self.handle_command(received, client_socket)
            

    def handle_command(self, opcion, client_socket):
        # Completar
        pass

    def actualizar_info(self):
        self.info = {'0' : '\n Menú de opciones \n' \
                            '[1] Casos en Chile \n' \
                            '[2] Casos en el Mundo \n' \
                            '[3] Consejos contra el Coronavirus \n' \
                            '[4] Reportar un nuevo caso',
                     '1' : f' Total (new) cases  \n Chile \n {self.casos_chile} ' \
                           f'confirmed cases ({self.nuevo_caso_chile}) \n 2264 deaths (627)',
                     '2' : f' Total (new) cases  \n Global \n {self.casos_mundo} ' \
                           f'confirmed cases ({self.nuevo_caso_mundo}) \n 404396 deaths (3539)',
                     '3' : 'Consejos \n' \
                            ' - Wash your hands frequently \n' \
                            ' - Avoid touching your eyes, mouth and nose \n' \
                            ' - Cover your mouth and nose with your bent elbow or tissue when you cough or sneeze \n' \
                            ' - Avoid crowded places \n' \
                            ' - Stay at home if you feel unwell - even with a slight fever and cough \n' \
                            ' - If you have a fever, cough and difficulty breathing, seek medical care early - but call by phone first',
                     '4' : 'Se reportó un nuevo caso en Chile '
                }
    def actualizar_casos(self):
        pass


if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    server = Bot(port, host)

In [ ]:
import socket
import threading


class Usuario:

    def __init__(self, port, host):

        self.host = host
        self.port = port
        self.socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        try:
            self.connect_to_server()
            self.listen()
            self.repl()
        except ConnectionError:
            print("Conexión terminada.")
            self.socket_client.close()
            exit()

    def connect_to_server(self):

        self.socket_client.connect((self.host, self.port))

    def listen(self):


        thread = threading.Thread(target=self.listen_thread, daemon=True)
        thread.start()

    def send(self, msg):

        msg_bytes = msg.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        self.socket_client.sendall(msg_length + msg_bytes)

    def listen_thread(self):
        while True:
            response_bytes_length = self.socket_client.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(self.socket_client.recv(read_length))

            print(f"{response.decode()}\n>>> ", end='')

    def repl(self):


        print("------ Whatsapp WHO ------\n Apreta 0 para recibir el menú de opciones\n>>> ", end='')
        while True:
            msg = input()
            response = self.send(msg)



if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    client = Usuario(port, host)

### Ejercicio 2: `Opción actualizar casos`

Ahora que implementaste `handle_command`, notas que cuando reportas un nuevo caso, no se contabiliza en la base de datos ni le avisa a otras personas conectadas. Para ello, vas a tener que modificar `handle_command` e implementar `sendall` y `actualizar_casos`. La idea es que cuando algún usuario reporte un nuevo contagiado en Chile, el método `actualizar_casos` actualice en el servidor la cantidad de contagiados del día y en total en Chile y en el mundo. Después a través del método `sendall` se deberá enviar el aviso a todos los usuarios conectados que hay un nuevo contagiado (piensa en alguna forma de guardar a los sockets cuando se conectan. Para este ejercicio puedes asumir que una vez que un usuario conectado nunca se desconectará). Si algún usuario quiere visualizar la información de los contagiados esta debe estar actualizada correctamente. Reutiliza el código en 1.1.